In [5]:
# ##############################################################################
#
# 통합 펩타이드 발굴 파이프라인 (Single-Notebook Peptide Discovery Pipeline)
#
# 이 코드는 Google Colab Pro/Pro+ 환경에서 실행하는 것을 권장합니다.
# Colab 메뉴에서 '런타임' -> '런타임 유형 변경'을 선택하여
# 하드웨어 가속기를 'GPU'로, 런타임 구성을 '높은 RAM'으로 설정해주세요.
#
# ##############################################################################


# ==============================================================================
# STEP 0: 환경 설정 및 모든 필수 라이브러리 설치
# ==============================================================================
# 이 셀은 파이프라인에 필요한 모든 소프트웨어를 설치합니다.
# ColabFold, Transformers, 결합력 평가 도구 등이 포함됩니다.
# 최초 실행 시 약 15~20분 정도 소요될 수 있습니다.

print("="*80)
print("STEP 0: 환경 설정 및 모든 필수 라이브러리 설치 (약 15-20분 소요)")
print("="*80)

import os
import sys
import site

# ❗ 추가된 부분: 한국 시간대(KST) 처리를 위한 pytz 라이브러리 설치
print("\n   > 시간대 처리 라이브러리 (pytz) 설치 중...")
os.system("pip install -q pytz")
print("   > pytz 설치 완료")

# ColabFold (AlphaFold2) 설치
print("\n[1/4] ColabFold (AlphaFold2) 설치 중...")
# ❗ 안정성 강화: ColabFold 설치 전, 충돌을 유발할 수 있는 기존 TensorFlow 패키지를 모두 제거합니다.
print("   > 기존 TensorFlow 패키지를 제거하여 충돌을 방지합니다...")
os.system("pip uninstall -y tensorflow tensorboard tb-nightly tensorflow-estimator tensorflow-hub tensorflow-io > /dev/null 2>&1")
os.system("pip install -q --no-warn-conflicts 'colabfold[alphafold] @ git+https://github.com/sokrypton/ColabFold'")
os.system("pip install -q --no-warn-conflicts 'jax[cuda11_pip]' -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html")

# ❗ 안정성 강화: 설치된 ColabFold 스크립트를 직접 수정하여 TensorFlow 오류를 영구적으로 방지합니다.
print("\n   > ColabFold 스크립트 패치 적용 중...")
try:
    dist_packages_path = site.getsitepackages()[0]
    batch_py_path = os.path.join(dist_packages_path, 'colabfold', 'batch.py')
    if os.path.exists(batch_py_path):
        os.system(f"sed -i 's/tf.get_logger().setLevel(logging.ERROR)/#tf.get_logger().setLevel(logging.ERROR)/g' {batch_py_path}")
        os.system(f"sed -i \"s/tf.config.set_visible_devices(\\[\\], 'GPU')/#tf.config.set_visible_devices(\\[\\], 'GPU')/g\" {batch_py_path}")
        print("   > 패치 적용 완료.")
    else:
        print(f"   > 경고: {batch_py_path}를 찾을 수 없어 패치를 건너뜁니다.")
except Exception as e:
    print(f"   > 경고: ColabFold 패치 중 오류 발생 - {e}")

# 펩타이드 생성 모델 관련 라이브러리 설치
print("\n[2/4] 펩타이드 생성 관련 라이브러리 (Transformers) 설치 중...")
os.system("pip install -q --upgrade transformers sentencepiece")

# 결합력 평가 도구 (Open Babel, PLIP, Pafnucy) 설치
print("\n[3/4] 결합력 평가 도구 (Open Babel, PLIP, Pafnucy) 설치 중...")
os.system("apt-get update -qq > /dev/null 2>&1")
# ❗ 안정성 강화: python3-openbabel을 함께 설치하여 파이썬 바인딩 문제를 해결합니다.
os.system("apt-get install -y --quiet openbabel python3-openbabel")
print("   > Open Babel 설치 완료")
print("   > PLIP 설치 중...")
os.system("pip install -q plip")
print("   > PLIP 설치 완료")


print("   > Pafnucy 및 ODDT 설치 중...")

# 1단계: ODDT 먼저 설치 (Pafnucy의 의존성)
os.system("pip install -q oddt")
print("   > ODDT 설치 완료")

# 2단계: 개선된 Pafnucy 설치 프로세스
def install_pafnucy_robust():
    """강화된 Pafnucy 설치 함수"""
    try:
        # 기존 pafnucy 폴더 완전 제거
        if os.path.exists("pafnucy"):
            os.system("rm -rf pafnucy")

        # Git clone 재시도
        print("   > Pafnucy 저장소 클론 중...")
        result = os.system("git clone https://github.com/oddt/pafnucy.git > /dev/null 2>&1")

        if result != 0 or not os.path.exists("pafnucy"):
            print("   > ⚠️ Git clone 실패, 대체 방법 시도...")
            # 대체: wget으로 zip 다운로드
            os.system("wget -q https://github.com/oddt/pafnucy/archive/refs/heads/master.zip")
            os.system("unzip -q master.zip")
            if os.path.exists("pafnucy-master"):
                os.system("mv pafnucy-master pafnucy")

        if os.path.exists("pafnucy"):
            print("   > Pafnucy 폴더 확인됨")

            # 현재 디렉터리 저장
            original_dir = os.getcwd()
            pafnucy_path = os.path.abspath("pafnucy")

            try:
                # pafnucy 경로를 Python path에 추가
                if pafnucy_path not in sys.path:
                    sys.path.insert(0, pafnucy_path)

                # 필요한 __init__.py 파일들 생성
                init_files = [
                    os.path.join(pafnucy_path, "__init__.py"),
                    os.path.join(pafnucy_path, "models", "__init__.py")
                ]

                for init_file in init_files:
                    os.makedirs(os.path.dirname(init_file), exist_ok=True)
                    if not os.path.exists(init_file):
                        with open(init_file, "w") as f:
                            f.write("# Pafnucy package\n")

                # requirements.txt가 있으면 설치
                req_file = os.path.join(pafnucy_path, "requirements.txt")
                if os.path.exists(req_file):
                    os.system(f"pip install -q -r {req_file}")

                print("   > Pafnucy 설정 완료")
                return True

            except Exception as e:
                print(f"   > Pafnucy 설정 중 오류: {e}")
                return False
        else:
            print("   > ❌ Pafnucy 폴더를 생성할 수 없음")
            return False

    except Exception as e:
        print(f"   > ❌ Pafnucy 설치 실패: {e}")
        return False

# Pafnucy 설치 실행
pafnucy_success = install_pafnucy_robust()

if pafnucy_success:
    print("   > ✅ Pafnucy 설치 및 설정 완료")
else:
    print("   > ⚠️ Pafnucy 설치 실패 - 파이프라인은 대체 함수로 계속 진행됩니다")

# 전역 변수로 Pafnucy 사용 가능 여부 설정
PAFNUCY_AVAILABLE = pafnucy_success

# ❗ 추가된 부분: Excel 파일 출력을 위한 openpyxl 라이브러리 설치
print("\\n   > Excel 파일 지원 라이브러리 (openpyxl) 설치 중...")
os.system("pip install -q openpyxl")
print("   > openpyxl 설치 완료")

# AutoDock Vina 다운로드
print("\n[4/4] AutoDock Vina 다운로드 중...")
if not os.path.exists("vina_1.2.3_linux_x86_64"):
    os.system("wget -q https://github.com/ccsb-scripps/AutoDock-Vina/releases/download/v1.2.3/vina_1.2.3_linux_x86_64.zip")
    os.system("unzip -q -o vina_1.2.3_linux_x86_64.zip")
    os.system("chmod +x vina_1.2.3_linux_x86_64/vina")

print("\n모든 설치 완료!")
print("="*80)
print("✅ STEP 0: 환경 설정이 성공적으로 완료되었습니다.")
print("="*80)


# ==============================================================================
# STEP 1: 파이프라인 실행을 위한 변수 설정
# ==============================================================================
# 이 셀에서 전체 파이프라인의 작동 방식을 제어하는 주요 변수들을 설정합니다.

print("\n" + "="*80)
print("STEP 1: 파이프라인 실행을 위한 변수 설정")
print("="*80)

import torch
from datetime import datetime
import pytz

# --- ❗ 사용자 설정 영역 ❗ ---

# 1. 생성할 펩타이드 후보의 개수
N_PEPTIDES = 5

# 2. 타겟 단백질의 아미노산 서열 (FASTA 형식, 한 줄로 입력)
# 예시: "PIAQIHILEGRSDEQKETLIREVSEAISRSLDAPLTSVRVIITEMAKGHFGIGGELASK"
TARGET_PROTEIN_SEQUENCE = "PIAQIHILEGRSDEQKETLIREVSEAISRSLDAPLTSVRVIITEMAKGHFGIGGELASK"

# 3. 생성할 펩타이드의 길이
PEPTIDE_LENGTH = 10

# 4. 결과 폴더의 기본 이름 접두사
BASE_FOLDER_PREFIX = "PDP"

# --- ❗ 수정된 부분: 한국 시간(KST)을 기준으로 동적 폴더 및 파일 이름 생성 ---
kst = pytz.timezone('Asia/Seoul')
now_kst = datetime.now(kst)
timestamp = now_kst.strftime("%Y%m%d_%H%M%S")

# 최종 결과 폴더명 (예: PDP_20231027_153000)
JOB_NAME = f"{BASE_FOLDER_PREFIX}_{timestamp}"

# --- 설정값 확인 및 디렉토리/파일 경로 생성 ---
os.makedirs(JOB_NAME, exist_ok=True)
PROTEIN_FASTA_PATH = os.path.join(JOB_NAME, "target_protein.fasta")

# ❗ 수정된 부분: 최종 결과 파일 경로만 동적으로 정의
OUTPUT_FINAL_XLSX_PATH = os.path.join(JOB_NAME, f"final_peptide_ranking_{timestamp}.xlsx")


with open(PROTEIN_FASTA_PATH, "w") as f:
    f.write(f">target_protein\n{TARGET_PROTEIN_SEQUENCE}\n")

print(f"✔️ 작업 폴더: {JOB_NAME}")
print(f"✔️ 생성할 펩타이드 개수: {N_PEPTIDES}")
print(f"✔️ 타겟 단백질 서열 길이: {len(TARGET_PROTEIN_SEQUENCE)}")
print(f"✔️ 생성할 펩타이드 길이: {PEPTIDE_LENGTH}")
print(f"✔️ 타겟 단백질 FASTA 파일 저장: {PROTEIN_FASTA_PATH}")
print(f"✔️ 최종 결과 파일 저장 경로: {OUTPUT_FINAL_XLSX_PATH}")
print("="*80)
print("✅ STEP 1: 설정이 완료되었습니다.")
print("="*80)


# ==============================================================================
# STEP 2: PepMLM (ESM-2)을 이용한 타겟 특이적 펩타이드 후보 생성
# ==============================================================================
print("\n" + "="*80)
print("STEP 2: PepMLM (ESM-2)을 이용한 타겟 특이적 펩타이드 후보 생성")
print("="*80)

import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForMaskedLM

# ESM-2 모델 및 토크나이저 로드
model_name = "facebook/esm2_t12_35M_UR50D"
print(f"'{model_name}' 모델과 토크나이저를 로딩합니다...")
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForMaskedLM.from_pretrained(model_name).to("cuda" if torch.cuda.is_available() else "cpu")
print("모델 로딩 완료!")

# 생성 파라미터
temperature = 1.0
top_k = 50

# 모델 입력용 프롬프트 생성 ("빈칸 채우기" 방식)
formatted_target = " ".join(list(TARGET_PROTEIN_SEQUENCE))
mask_tokens = " ".join([tokenizer.mask_token] * PEPTIDE_LENGTH)
prompt = f"{tokenizer.cls_token} {formatted_target} {tokenizer.eos_token} {mask_tokens}"
input_ids = tokenizer(prompt, return_tensors="pt").input_ids

mask_token_indices = (input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]

peptides = []
peptide_fasta_paths = []

print("\n펩타이드 서열 생성을 시작합니다 (반복적 마스크 채우기 방식)...")
with torch.no_grad():
    for i in range(N_PEPTIDES):
        current_ids = input_ids.clone().to(model.device)
        shuffled_mask_indices = mask_token_indices[torch.randperm(len(mask_token_indices))]

        for mask_idx in shuffled_mask_indices:
            outputs = model(input_ids=current_ids)
            logits = outputs.logits
            mask_logits = logits[0, mask_idx, :]
            filtered_logits = mask_logits / temperature
            effective_top_k = min(top_k, tokenizer.vocab_size)
            top_k_values, top_k_indices = torch.topk(filtered_logits, effective_top_k)
            filter_tensor = torch.full_like(filtered_logits, -float('Inf'))
            filter_tensor.scatter_(0, top_k_indices, top_k_values)
            probs = F.softmax(filter_tensor, dim=-1)
            predicted_token_id = torch.multinomial(probs, num_samples=1)
            current_ids[0, mask_idx] = predicted_token_id.item()

        generated_token_ids = current_ids[0, mask_token_indices]
        sequence_part = tokenizer.decode(generated_token_ids, skip_special_tokens=True)
        sequence = "".join(sequence_part.split())
        peptides.append(sequence)

        fasta_path = os.path.join(JOB_NAME, f"peptide_{i}.fasta")
        with open(fasta_path, "w") as f:
            f.write(f">peptide_{i}\n{sequence}\n")
        peptide_fasta_paths.append(fasta_path)
        print(f"  [{i+1}/{N_PEPTIDES}] 생성 완료: {sequence} (길이: {len(sequence)})")

print("\n--- 생성된 펩타이드 후보 목록 ---")
for i, seq in enumerate(peptides):
    print(f"  - 후보 {i}: {seq}")
print("="*80)
print(f"✅ STEP 2: 총 {N_PEPTIDES}개의 펩타이드 후보 생성을 완료했습니다.")
print("="*80)


# ==============================================================================
# STEP 3: 단백질-펩타이드 복합체 3D 구조 예측 (수정된 ColabFold 실행)
# ==============================================================================
import glob

print("\n" + "="*80)
print("STEP 3: 단백질-펩타이드 복합체 3D 구조 예측 (수정된 ColabFold 실행)")
print("="*80)

predicted_pdb_files = []

# ❗ 효율성 개선: 모든 복합체를 하나의 CSV 파일로 만들어 배치 처리합니다.
print("\n배치 처리를 위한 복합체 CSV 파일 생성 중...")
batch_csv_path = os.path.join(JOB_NAME, "batch_complexes.csv")
with open(batch_csv_path, "w") as f:
    f.write("id,sequence\n")
    for i in range(N_PEPTIDES):
        peptide_seq = peptides[i]
        complex_sequence = f"{TARGET_PROTEIN_SEQUENCE}:{peptide_seq}"
        f.write(f"complex_{i},{complex_sequence}\n")

print(f"✅ 배치 파일 생성 완료: {batch_csv_path}")

# ColabFold 배치 실행
output_dir = os.path.join(JOB_NAME, "colabfold_batch_output")
os.makedirs(output_dir, exist_ok=True)
log_file = os.path.join(output_dir, "colabfold_batch.log")

print(f"\nColabFold 배치 실행 시작... (출력 디렉토리: {output_dir})")
print("⏰ 예상 소요 시간: 10-30분 (복합체 개수에 따라 달라집니다)")

# ❗ 안정성 강화: Colab 환경에 최적화된 옵션을 사용합니다.
colabfold_cmd = (f"colabfold_batch "
                f"--num-recycle 1 "
                f"--model-type alphafold2_multimer_v3 "
                f"--rank ptm "
                f"--max-msa 32:128 "
                f"--num-models 1 "
                f"--stop-at-score 0.5 "
                f"{batch_csv_path} {output_dir} > {log_file} 2>&1")

print(f"실행 명령어: {colabfold_cmd}")
result = os.system(colabfold_cmd)

# 결과 확인
print(f"\nColabFold 실행 완료 (종료 코드: {result})")

# 생성된 PDB 파일 찾기
for i in range(N_PEPTIDES):
    pdb_pattern = os.path.join(output_dir, f"complex_{i}_unrelaxed_rank_001*.pdb")
    pdb_files = sorted(glob.glob(pdb_pattern))

    if pdb_files:
        predicted_pdb_files.append(pdb_files[0])
        print(f"  ✅ 복합체 {i}: {os.path.basename(pdb_files[0])}")
    else:
        print(f"  ❌ 복합체 {i}: PDB 파일을 찾을 수 없음")

# 실패 시 로그 파일 내용 출력
if len(predicted_pdb_files) < N_PEPTIDES and os.path.exists(log_file):
    print("\n" + "="*50)
    print("⚠️ 일부 예측이 실패했습니다. COLABFOLD 실행 로그:")
    print("="*50)
    with open(log_file, 'r') as f:
        print(f.read()[-2000:])
    print("="*50)

print("="*80)
print(f"✅ STEP 3: 총 {len(predicted_pdb_files)}개의 3D 구조 예측을 완료했습니다.")
print("="*80)


# ==============================================================================
# STEP 3.5: 구조 예측 신뢰도 점수(pTM) 확인 및 저장
# ==============================================================================
import json
import pandas as pd
from IPython.display import display
import re

print("\n" + "="*80)
print("STEP 3.5: 구조 예측 신뢰도 점수(pTM) 확인")
print("="*80)

scores_info = []
ptm_scores_map = {} # ❗ 추가된 부분: 펩타이드 서열을 키로 pTM 점수를 저장할 딕셔너리

# 다양한 점수 파일 패턴 시도
score_file_patterns = [
    os.path.join(output_dir, "*_scores.json"),
    os.path.join(output_dir, "complex_*_scores.json"),
    os.path.join(output_dir, "*_rank_001_*.json"),
    os.path.join(output_dir, "*_score*.json")
]

all_score_files = []
for pattern in score_file_patterns:
    files = sorted(glob.glob(pattern))
    all_score_files.extend(files)

# 중복 제거
all_score_files = list(set(all_score_files))

print(f"찾은 점수 파일들: {len(all_score_files)}개")

if not all_score_files:
    print("⚠️ ColabFold 점수 파일을 찾을 수 없습니다. pTM 점수는 0으로 처리됩니다.")
else:
    print(f"총 {len(all_score_files)}개의 점수 파일을 분석합니다...")

    for score_file in all_score_files:
        try:
            basename = os.path.basename(score_file)
            match = re.search(r'complex_(\d+)', basename)
            if not match:
                continue

            peptide_index = int(match.group(1))
            with open(score_file, 'r') as f:
                data = json.load(f)

            ptm_score = data.get('ptm', data.get('iptm', 0.0))
            if isinstance(ptm_score, list):
                ptm_score = ptm_score[0] if ptm_score else 0.0

            if peptide_index < len(peptides):
                peptide_seq = peptides[peptide_index]
                ptm_scores_map[peptide_seq] = round(float(ptm_score), 3)
                print(f"  복합체 {peptide_index} ({peptide_seq}): pTM = {ptm_scores_map[peptide_seq]}")

        except Exception as e:
            print(f"오류: {score_file} 처리 중 문제 발생 - {e}")
            continue

print("\n" + "="*80)
print("✅ STEP 3.5: pTM 점수 확인이 완료되었습니다.")
print("="*80)


# ==============================================================================
# STEP 4: 수정된 결합력 평가 및 최종 랭킹 계산 (Vina 경로 문제 해결)
# ==============================================================================
print("\n" + "="*80)
print("STEP 4: 수정된 결합력 평가 및 최종 랭킹 계산")
print("="*80)

import re
import subprocess
import glob
from oddt import toolkit

# ❗ 수정된 부분: Pafnucy 안전한 임포트
try:
    if 'PAFNUCY_AVAILABLE' in globals() and PAFNUCY_AVAILABLE:
        from pafnucy.models import pafnucy_de_ensemble_d as pafnucy_model
        print("✅ Pafnucy 모듈 로드 성공")
    else:
        raise ImportError("Pafnucy not available")
except ImportError:
    PAFNUCY_AVAILABLE = False
    pafnucy_model = None
    print("⚠️ Pafnucy 모듈 없음 - 대체 함수 사용")

# 대체 함수 정의
def predict_pafnucy_affinity_fallback(receptor_pdb, ligand_pdb):
    """Pafnucy 모듈이 없을 때 사용하는 대체 함수"""
    print("    -> Pafnucy 모듈이 없어 간단한 추정치를 사용합니다...")
    try:
        receptor_coords, ligand_coords = [], []

        # Receptor coordinates
        with open(receptor_pdb, 'r') as f:
            for line in f:
                if line.startswith(('ATOM', 'HETATM')):
                    receptor_coords.append((
                        float(line[30:38]),
                        float(line[38:46]),
                        float(line[46:54])
                    ))

        # Ligand coordinates
        with open(ligand_pdb, 'r') as f:
            for line in f:
                if line.startswith(('ATOM', 'HETATM')):
                    ligand_coords.append((
                        float(line[30:38]),
                        float(line[38:46]),
                        float(line[46:54])
                    ))

        if not receptor_coords or not ligand_coords:
            return 0.0

        # 최소 거리 및 접촉 수 계산
        min_distance = float('inf')
        close_contacts = 0

        for lx, ly, lz in ligand_coords:
            for rx, ry, rz in receptor_coords:
                distance = ((lx-rx)**2 + (ly-ry)**2 + (lz-rz)**2)**0.5
                min_distance = min(min_distance, distance)
                if distance < 4.0:
                    close_contacts += 1

        # 친화도 추정 (-logKi 스케일)
        if min_distance < float('inf'):
            affinity = max(8.0 - min_distance, 0.0) + (close_contacts * 0.05)
            print(f"       간단 추정 친화도: {affinity:.3f} (-logKi)")
            return affinity

    except Exception as e:
        print(f"       친화도 추정 오류: {e}")

    return 0.0

# Vina 실행파일 경로 확인 및 설정
def find_vina_executable():
    """Vina 실행파일을 찾아서 경로 반환"""
    possible_paths = [
        "./vina_1.2.3_linux_x86_64/bin/vina",
        "./vina_1.2.3_linux_x86_64/vina",
        "vina",
        "/usr/local/bin/vina",
        "/opt/vina/bin/vina"
    ]
    for path in possible_paths:
        if os.path.exists(path):
            os.chmod(path, 0o755)
            return path
    return None

def setup_vina():
    """Vina 다운로드 및 설정"""
    print("Vina 설정 중...")
    if not os.path.exists("vina_1.2.3_linux_x86_64.zip"):
        os.system("wget -q https://github.com/ccsb-scripps/AutoDock-Vina/releases/download/v1.2.3/vina_1.2.3_linux_x86_64.zip")
    os.system("unzip -o -q vina_1.2.3_linux_x86_64.zip")
    vina_files = glob.glob("vina_1.2.3_linux_x86_64/**/vina", recursive=True)
    for vina_file in vina_files:
        os.chmod(vina_file, 0o755)
    return find_vina_executable()

def predict_pafnucy_affinity(receptor_pdb, ligand_pdb):
    """Pafnucy 모델 또는 대체 함수를 사용하여 결합 친화도를 예측합니다."""

    if PAFNUCY_AVAILABLE and pafnucy_model is not None:
        print("    -> Pafnucy 모델로 결합 친화도 예측 중...")
        try:
            receptor = next(toolkit.readfile('pdb', receptor_pdb))
            receptor.protein = True
            ligand = next(toolkit.readfile('pdb', ligand_pdb))

            affinity = pafnucy_model.predict_affinity(receptor, ligand)
            print(f"       Pafnucy 예측값 (-logKi): {affinity:.3f}")
            return affinity

        except Exception as e:
            print(f"       Pafnucy 예측 실패: {e}")
            return predict_pafnucy_affinity_fallback(receptor_pdb, ligand_pdb)
    else:
        return predict_pafnucy_affinity_fallback(receptor_pdb, ligand_pdb)

def estimate_binding_energy(receptor_pdb, ligand_pdb):
    """간단한 거리 기반 결합 에너지 추정"""
    try:
        receptor_coords, ligand_coords = [], []
        with open(receptor_pdb, 'r') as f:
            for line in f:
                if line.startswith(('ATOM', 'HETATM')):
                    receptor_coords.append((float(line[30:38]), float(line[38:46]), float(line[46:54])))
        with open(ligand_pdb, 'r') as f:
            for line in f:
                if line.startswith(('ATOM', 'HETATM')):
                    ligand_coords.append((float(line[30:38]), float(line[38:46]), float(line[46:54])))
        if not receptor_coords or not ligand_coords: return 0.0
        min_distance, close_contacts = float('inf'), 0
        for lx, ly, lz in ligand_coords:
            for rx, ry, rz in receptor_coords:
                distance = ((lx-rx)**2 + (ly-ry)**2 + (lz-rz)**2)**0.5
                min_distance = min(min_distance, distance)
                if distance < 4.0: close_contacts += 1
        if min_distance < float('inf'):
            return max(-1.0 * (10.0 / max(min_distance, 0.1)) - (close_contacts * 0.1), -15.0)
    except Exception as e:
        print(f"       거리 기반 추정 오류: {e}")
    return 0.0

def calculate_plip_interactions_simple(pdb_file):
    """간단한 거리 기반 상호작용 계산"""
    try:
        chain_a_coords, chain_b_coords = [], []
        with open(pdb_file, 'r') as f:
            for line in f:
                if line.startswith(('ATOM', 'HETATM')):
                    chain, atom_type = line[21], line[12:16].strip()
                    coords = (float(line[30:38]), float(line[38:46]), float(line[46:54]), atom_type)
                    if chain == 'A': chain_a_coords.append(coords)
                    elif chain == 'B': chain_b_coords.append(coords)
        h_bonds, hydrophobic = 0, 0
        for bx, by, bz, b_atom in chain_b_coords:
            for ax, ay, az, a_atom in chain_a_coords:
                distance = ((bx-ax)**2 + (by-ay)**2 + (bz-az)**2)**0.5
                if distance <= 3.5 and any(c in a_atom for c in 'NO') and any(c in b_atom for c in 'NO'): h_bonds += 1
                if distance <= 4.0 and 'C' in a_atom and 'C' in b_atom: hydrophobic += 1
        print(f"       간단한 상호작용 분석: H-bonds={h_bonds}, Hydrophobic={hydrophobic}")
        return h_bonds + hydrophobic
    except Exception as e:
        print(f"       간단한 상호작용 계산 오류: {e}")
        return 0

def split_pdb_and_get_center(pdb_file, base_name):
    """PDB 파일을 receptor(A)와 ligand(B)로 분리하고 ligand의 중심점을 계산"""
    receptor_file, ligand_file, coords = f"{base_name}_receptor.pdb", f"{base_name}_ligand.pdb", []
    print(f"    -> PDB 분리 중: {os.path.basename(pdb_file)}")
    try:
        with open(pdb_file, 'r') as f_in, open(receptor_file, 'w') as f_r, open(ligand_file, 'w') as f_l:
            for line in f_in:
                if line.startswith(("ATOM", "HETATM")):
                    if line[21] == 'A': f_r.write(line)
                    elif line[21] == 'B':
                        f_l.write(line)
                        coords.append((float(line[30:38]), float(line[38:46]), float(line[46:54])))
        if coords:
            center = (sum(c[0] for c in coords)/len(coords), sum(c[1] for c in coords)/len(coords), sum(c[2] for c in coords)/len(coords))
            return receptor_file, ligand_file, center
    except Exception as e:
        print(f"       오류: PDB 분리 실패 - {e}")
    return receptor_file, ligand_file, (0, 0, 0)

vina_executable = find_vina_executable() or setup_vina()
if vina_executable: print(f"✅ Vina 실행파일 경로: {vina_executable}")
else: print("⚠️ Vina를 찾을 수 없습니다. 간단한 추정 방법을 사용합니다.")

results = []
if not predicted_pdb_files:
    print("평가할 PDB 파일이 없습니다.")
else:
    print(f"총 {len(predicted_pdb_files)}개의 구조에 대해 평가를 시작합니다...")
    for idx, pred_pdb in enumerate(predicted_pdb_files):
        print(f"\n  평가 중 ({idx+1}/{len(predicted_pdb_files)}): {os.path.basename(pred_pdb)}")
        base_name = os.path.join(JOB_NAME, f"eval_{idx}")
        if not os.path.exists(pred_pdb) or os.path.getsize(pred_pdb) == 0: continue

        receptor_pdb, ligand_pdb, center = split_pdb_and_get_center(pred_pdb, base_name)
        vina_score = 0.0
        if vina_executable:
            try:
                receptor_pdbqt, ligand_pdbqt = f"{base_name}_receptor.pdbqt", f"{base_name}_ligand.pdbqt"
                os.system(f"obabel -ipdb {receptor_pdb} -opdbqt -O {receptor_pdbqt} >/dev/null 2>&1")
                os.system(f"obabel -ipdb {ligand_pdb} -opdbqt -O {ligand_pdbqt} >/dev/null 2>&1")
                if os.path.exists(receptor_pdbqt) and os.path.exists(ligand_pdbqt):
                    log_file = f"{base_name}_vina.log"
                    cmd = [vina_executable, "--receptor", receptor_pdbqt, "--ligand", ligand_pdbqt,
                           "--center_x", str(center[0]), "--center_y", str(center[1]), "--center_z", str(center[2]),
                           "--size_x", "20", "--size_y", "20", "--size_z", "20", "--exhaustiveness", "4", "--log", log_file]
                    subprocess.run(cmd, capture_output=True, text=True, timeout=300)
                    if os.path.exists(log_file):
                        with open(log_file, 'r') as f:
                            matches = re.findall(r'^\s*1\s+([-+]?\d+\.?\d*)', f.read(), re.MULTILINE)
                            if matches: vina_score = float(matches[0])
                if vina_score == 0.0: vina_score = estimate_binding_energy(receptor_pdb, ligand_pdb)
            except Exception:
                vina_score = estimate_binding_energy(receptor_pdb, ligand_pdb)
        else:
            vina_score = estimate_binding_energy(receptor_pdb, ligand_pdb)

        interaction_count = calculate_plip_interactions_simple(pred_pdb)
        # ❗ 수정된 부분: Vina 점수 기반 추정 대신 실제 Pafnucy 모델 예측 함수 호출
        pafnucy_affinity = predict_pafnucy_affinity(receptor_pdb, ligand_pdb)

        final_score = abs(vina_score) + pafnucy_affinity + (0.2 * interaction_count)

        try:
            peptide_index = int(os.path.basename(pred_pdb).split('_')[1])
            peptide_seq = peptides[peptide_index]
        except (IndexError, ValueError):
            peptide_seq = f"Unknown_{idx}"

        # ❗ 수정된 부분: 결과 딕셔너리에 ptm_scores_map에서 조회한 pTM 점수와 Pafnucy 점수 추가
        results.append({
            "Peptide Sequence": peptide_seq,
            "pTM Score": ptm_scores_map.get(peptide_seq, 0.0),
            "Vina Score (kcal/mol)": round(vina_score, 3),
            "Pafnucy Affinity (-logKi)": round(pafnucy_affinity, 3),
            "PLIP Interactions": interaction_count,
            "Final Score": round(final_score, 3),
            "Source PDB": os.path.basename(pred_pdb),
        })
        print(f"    -> 평가 완료: Final Score = {final_score:.2f}")

print("="*80)
print(f"✅ STEP 4: 모든 구조에 대한 평가 및 점수 계산을 완료했습니다.")
print("="*80)


# ==============================================================================
# STEP 5: 최종 결과 확인 및 저장
# ==============================================================================
print("\n" + "="*80)
print("STEP 5: 최종 결과 확인 및 저장")
print("="*80)

if results:
    import pandas as pd
    from IPython.display import display

    df = pd.DataFrame(results)

    # ❗ 수정된 부분: 최종 결과에 표시할 컬럼 순서에 Pafnucy 점수 추가
    column_order = [
        "Peptide Sequence", "Final Score", "pTM Score",
        "Vina Score (kcal/mol)", "Pafnucy Affinity (-logKi)",
        "PLIP Interactions", "Source PDB"
    ]
    # DataFrame에 모든 컬럼이 있는지 확인 후 순서 재배치
    df_sorted = df.sort_values("Final Score", ascending=False).reset_index(drop=True)
    df_final = df_sorted[[col for col in column_order if col in df_sorted.columns]]

    df_final.to_excel(OUTPUT_FINAL_XLSX_PATH, index=False)

    print("\n🏆 최종 펩타이드 후보 랭킹:")
    display(df_final)

    print(f"\n💾 전체 결과가 Excel 파일로 저장되었습니다: {OUTPUT_FINAL_XLSX_PATH}")
    print("   (Colab 왼쪽의 파일 탐색기에서 다운로드할 수 있습니다.)")
else:
    print("\n❌ 최종 결과가 없습니다. 파이프라인 중간에 오류가 발생했을 수 있습니다.")

print("="*80)
print("🎉 모든 파이프라인 실행이 완료되었습니다. 🎉")
print("="*80)


STEP 0: 환경 설정 및 모든 필수 라이브러리 설치 (약 15-20분 소요)

   > 시간대 처리 라이브러리 (pytz) 설치 중...
   > pytz 설치 완료

[1/4] ColabFold (AlphaFold2) 설치 중...
   > 기존 TensorFlow 패키지를 제거하여 충돌을 방지합니다...

   > ColabFold 스크립트 패치 적용 중...
   > 패치 적용 완료.

[2/4] 펩타이드 생성 관련 라이브러리 (Transformers) 설치 중...

[3/4] 결합력 평가 도구 (Open Babel, PLIP, Pafnucy) 설치 중...
   > Open Babel 설치 완료
   > PLIP 설치 중...
   > PLIP 설치 완료
   > Pafnucy 및 ODDT 설치 중...
   > ODDT 설치 완료
   > Pafnucy 저장소 클론 중...
   > ⚠️ Git clone 실패, 대체 방법 시도...
   > ❌ Pafnucy 폴더를 생성할 수 없음
   > ⚠️ Pafnucy 설치 실패 - 파이프라인은 대체 함수로 계속 진행됩니다
\n   > Excel 파일 지원 라이브러리 (openpyxl) 설치 중...
   > openpyxl 설치 완료

[4/4] AutoDock Vina 다운로드 중...

모든 설치 완료!
✅ STEP 0: 환경 설정이 성공적으로 완료되었습니다.

STEP 1: 파이프라인 실행을 위한 변수 설정
✔️ 작업 폴더: PDP_20250818_015124
✔️ 생성할 펩타이드 개수: 5
✔️ 타겟 단백질 서열 길이: 59
✔️ 생성할 펩타이드 길이: 10
✔️ 타겟 단백질 FASTA 파일 저장: PDP_20250818_015124/target_protein.fasta
✔️ 최종 결과 파일 저장 경로: PDP_20250818_015124/final_peptide_ranking_20250818_015124.xlsx
✅ STEP 1: 설정이 완료되었습니다.

STEP 2: PepMLM (ESM-2

,Peptide Sequence,Final Score,pTM Score,Vina Score (kcal/mol),Pafnucy Affinity (-logKi),PLIP Interactions,Source PDB
0,EPKDIETAGT,116.488,0.67,-15.000,39.288,311,complex_3_unrelaxed_rank_001_alphafold2_multim...
1,RSKPQPRVGE,95.747,0.68,-15.000,31.747,245,complex_2_unrelaxed_rank_001_alphafold2_multim...
2,TLAIAARMKR,87.919,0.65,-15.000,28.519,222,complex_4_unrelaxed_rank_001_alphafold2_multim...
3,VGIGLQKGVG,35.495,0.69,-15.000,11.095,47,complex_0_unrelaxed_rank_001_alphafold2_multim...
4,LARGKRKSHS,7.260,0.67,-2.848,4.412,0,complex_1_unrelaxed_rank_001_alphafold2_multim...



💾 전체 결과가 Excel 파일로 저장되었습니다: PDP_20250818_015124/final_peptide_ranking_20250818_015124.xlsx
   (Colab 왼쪽의 파일 탐색기에서 다운로드할 수 있습니다.)
🎉 모든 파이프라인 실행이 완료되었습니다. 🎉
